In [ ]:
from urllib.request import urlopen, Request

url = "https://db.netkeiba.com/race/202405050511"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"}

request = Request(url, headers=headers)
html = urlopen(request).read()

In [ ]:
html

In [ ]:
import pandas as pd
df = pd.read_html(html)
df

In [ ]:
df[0]

In [ ]:
df[1]

In [ ]:
df[2]

# 開催日一覧を取得
```
https://race.netkeiba.com/top/calendar.html?pid=race_calendar&year=2023
https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105
```

In [ ]:
url = "https://race.netkeiba.com/top/calendar.html?pid=race_calendar&year=2023"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"}
request = Request(url, headers=headers)
html = urlopen(request).read()
html

In [ ]:
# htmlから要素を取り出したい時はBeutifulsoupをつかう 

from bs4 import BeautifulSoup

soup = BeautifulSoup(html)
soup

In [ ]:
# aタグのhref要素から開催日を取り出す
a_tag = soup.find("table", class_="Calendar_Table").find("a")
a_tag["href"]

In [ ]:
# 開催日を抽出 
import re
re.findall(r"kaisai_date=(\d{8})", a_tag["href"])[0]

In [ ]:
# a_tag_list = soup.find("table", class_="Calendar_Table").find_all("a")
# kaisai_date_list = []
# for a_tag in a_tag_list:
#   kaisai_date = re.findall(r"kaisai_date=(\d{8})", a_tag["href"])[0]
#   kaisai_date_list.append(kaisai_date)

# kaisai_date_list


In [ ]:
def url_open_with_headers(url):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"}
  request = Request(url, headers=headers)
  html = urlopen(request).read()
  return html

In [ ]:
import time
from tqdm import tqdm

def scrape_kaisai_date(from_, to_):
  kaisai_date_list = []
  for date in tqdm(pd.date_range(from_, to_, freq="MS")):
    year, month = date.year, date.month
    url = f"https://race.netkeiba.com/top/calendar.html?pid=race_calendar&year={year}&month={month}"
    html = url_open_with_headers(url)
    time.sleep(1) # 忘れない
    soup = BeautifulSoup(html)
    a_tag_list = soup.find("table", class_="Calendar_Table").find_all("a")
    
    for a_tag in a_tag_list:
      kaisai_date = re.findall(r"kaisai_date=(\d{8})", a_tag["href"])[0]
      kaisai_date_list.append(kaisai_date)

  return kaisai_date_list


In [ ]:
# from_, to_ = "2023-01", "2023-12"

# kaisai_date_list_at_2023 = scrape_kaisai_date("2023-01", "2023-12")
# kaisai_date_list_at_2023

## 開催ページからIDを取得

In [ ]:
url = "https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105"

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
html = url_open_with_headers(url)
soup = BeautifulSoup(html, features="lxml")

In [ ]:
soup.find("div", class_="RaceList_Box")

取得できないときはselenium, Chrome Driverを使用

In [ ]:
# pip install selenium
# pip install webdriver-manager

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [ ]:
# chromeのversionにあったdriverを準備する→chromeを使う前提
driver_path = ChromeDriverManager().install()
driver = webdriver.Chrome(service=Service(driver_path))

In [ ]:
driver.get(url)

In [ ]:
from selenium.webdriver.common.by import By

li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
li_list

In [ ]:
li_0 = li_list[0]
li_0

In [ ]:
href = li_0.find_element(By.TAG_NAME, "a").get_attribute("href")
href

In [ ]:
import re
rece_id = re.findall(r"race_id=(\d{12})", href)[0]
rece_id

In [ ]:
race_id_list = []
for li in li_list:
  href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
  race_id = re.findall(r"race_id=(\d{12})" ,href)[0]
  race_id_list.append(race_id)

race_id_list

In [ ]:
len(race_id_list)

In [ ]:
driver.quit()

In [ ]:
from src.scraping import scrape_kaisai_date

In [ ]:
race_date_list = scrape_kaisai_date(from_="2023-01", to_="2023-12")
race_date_list

In [ ]:
import time
from tqdm import tqdm
import traceback
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


def scrape_race_ids(race_date_list: list[str]):
  driver_options = Options()
  driver_options.add_argument("--headless") # ヘッドレスで実行
  driver_path = ChromeDriverManager().install()
  race_id_list = []
  with webdriver.Chrome(service=Service(driver_path), options=driver_options) as driver:
  # driver = webdriver.Chrome(service=Service(driver_path), options=driver_options)
    for race_date in tqdm(race_date_list):
      url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={race_date}"
      try:
        driver.get(url)
        time.sleep(1) # 忘れない
        li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
        
        for li in li_list:
          href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
          race_id = re.findall(r"race_id=(\d{12})" ,href)[0]
          race_id_list.append(race_id)
      except:
        print("stop at", url)
        print(traceback.format_exc())
        break
    
  # driver.quit()
  return race_id_list

In [ ]:
from src.scraping import scrape_kaisai_date
race_date_list = race_date_list = scrape_kaisai_date(from_="2023-01", to_="2023-02")
race_id_list = scrape_race_ids(race_date_list)

race_id_list

### レース詳細画面からテーブルを取得

In [ ]:
from src.scraping import url_open_with_headers


In [ ]:
url = "https://db.netkeiba.com/race/202306010101"
html = url_open_with_headers(url)

In [ ]:
html

In [ ]:
import pandas as pd

In [ ]:
table = pd.read_html(html)
table[0]

In [ ]:
import pickle

with open("src/race_id_list_2023.pkl", "rb") as f:
  race_id_list = pickle.load(f)
  

In [ ]:
len(race_id_list)

In [ ]:
# import time

# for race_id in race_id_list[:4]:
#   url = f"https://db.netkeiba.com/race/{race_id}"
#   html = url_open_with_headers(url)
#   time.sleep(1)
#   df = pd.read_html(html)[0]



`df`を`dump`するの前に、後々、データ追加したくなる時用にHTML自体を保存する

In [ ]:
html

In [ ]:
with open("data/html/202306010101.bin", "wb") as f:
  f.write(html)

In [ ]:
from pathlib import Path

HTML_DIR = Path("data", "html")

In [ ]:
with open(HTML_DIR / "202306010101.bin", "wb") as f:
  f.write(html)

In [ ]:
import time
from pathlib import Path
from tqdm import tqdm

HTML_DIR = Path("data", "html")

def scrape_race_detail_html(race_id_list: list[str], save_dir: Path) -> list[Path]:
  html_path_list = []
  for race_id in tqdm(race_id_list):
    # file exists check
    filepath = save_dir / f"{race_id}.bin"
    if filepath.is_file():
      print("skipped", filepath)
      continue
    
    # scraping
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = url_open_with_headers(url)
    time.sleep(1)
    
    # write html
    with open(filepath, "wb") as f:
      f.write(html)
    html_path_list.append(filepath)
  
  return html_path_list

In [ ]:
html_path_list = scrape_race_detail_html(race_id_list[:12], HTML_DIR)
html_path_list

In [ ]:
import os
from src.constants import PROJECT_ROOT, RACE_DETAIL_HTML_DIR

os.chdir(PROJECT_ROOT)
print("Current working directory:", os.getcwd())

In [ ]:
print(RACE_DETAIL_HTML_DIR)

In [ ]:
html_path_list = list(RACE_DETAIL_HTML_DIR.glob("*"))

In [ ]:
len(html_path_list)

In [ ]:
dfs = {}
for html_path in html_path_list[:3]:
  with open(html_path, "rb") as f:
    race_id = html_path.stem
    html = f.read()
    df = pd.read_html(html)[0]
    df.index = [race_id] * len(df) # race_idをindexに(list * int = 繰り返しになる)
    dfs[race_id] = df
    
# valueだけ繋げる
concat_df = pd.concat(dfs.values())
concat_df.index.name = "race_id"


In [ ]:
concat_df

In [ ]:
def create_results(html_path_list: list[Path]) -> pd.DataFrame:
  dfs = {}
  for html_path in tqdm(html_path_list):
    with open(html_path, "rb") as f:
      race_id = html_path.stem
      html = f.read()
      df = pd.read_html(html)[0]
      df.index = [race_id] * len(df) # race_idをindexに(list * int = 繰り返しになる)
      dfs[race_id] = df
    
  # valueだけ繋げる
  concat_df = pd.concat(dfs.values())
  concat_df.index.name = "race_id"
  return concat_df

In [ ]:
results = create_results(html_path_list)

In [ ]:
results

## ウマの過去成績テーブルの作成

In [ ]:
# ウマの過去成績ページ
url = "https://db.netkeiba.com/horse/2020103575"

In [ ]:
from src.scraping import url_open_with_headers

html = url_open_with_headers(url)
html

In [ ]:
import pandas as pd

pd.read_html(html)[2]

#### raceに紐づくウマIDを取得する

In [ ]:
# 対象レースのURL
race_url = "https://db.netkeiba.com/race/202306010101/"
html = url_open_with_headers(race_url)
html

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "lxml")
table = soup.find("table", class_="race_table_01 nk_tb_common")

print(table)

#### aタグの中でhorseを含むものだけにを絞り込む

In [ ]:
import re
# ^は前方一致
a_list = table.find_all("a", href=re.compile(r"^/horse/"))

URLからrace idを取り出す

In [ ]:
example_url = a_list[0]["href"]
horse_id = re.findall(r"\d{10}", example_url)[0]
horse_id

In [ ]:
from tqdm import tqdm

horse_id_list = []
for a in tqdm(a_list):
  horse_id = re.findall(r"\d{10}", a["href"])[0]
  horse_id_list.append(horse_id)

In [ ]:
len(horse_id_list)

In [ ]:
# 関数化テスト
from src.create_rawdf import create_race_results
from src.constants import RACE_DETAIL_HTML_DIR, RACE_RESULTS_RAWDF_DIR

html_path_list_2023 = list(RACE_DETAIL_HTML_DIR.glob("*.bin"))
results = create_race_results(html_path_list=html_path_list_2023[:10], 
                         save_dir=RACE_RESULTS_RAWDF_DIR, 
                         save_filename="results_2023.csv")

In [ ]:
# results

In [ ]:
# results.isnull().sum()

resultsを保存しておく

In [ ]:
# 関数に組み込む
# from src.constants import RACE_RESULTS_RAWDF_DIR

# results.to_csv(RACE_RESULTS_RAWDF_DIR / "results_2023.csv", sep="\t")

### ウマID一覧を取得する

In [ ]:
horse_id_list = results["horse_id"].unique()
len(horse_id_list)

### 馬の過去成績テーブルを作成する

In [ ]:
from src.constants import HORSE_DETAIL_HTML_DIR
from src.scraping import scrape_horse_detail_html

html_path_horse = scrape_horse_detail_html(horse_id_list=horse_id_list)

In [ ]:
from src.constants import HORSE_DETAIL_HTML_DIR, RACE_RESULTS_RAWDF_DIR
html_path_horse = list(HORSE_DETAIL_HTML_DIR.glob("*.bin"))
len(html_path_horse)

In [ ]:
from src.create_rawdf import create_horse_results

horse_results = create_horse_results(html_path_list=html_path_horse, save_dir=RACE_RESULTS_RAWDF_DIR, save_filename="horse_results_2023")